In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

import scipy as sp
from sklearn.linear_model import LinearRegression

pd.set_option("display.max_columns", 100)

import statsmodels.api as sm
import statsmodels.stats.api as sms
from scipy import stats


In [2]:
df = pd.read_csv('Ames_HousePrice.csv')
df.shape

(2580, 82)

In [3]:
df.isna().sum(axis=0).sort_values(ascending=False).head(30)

PoolQC           2571
MiscFeature      2483
Alley            2412
Fence            2055
FireplaceQu      1241
LotFrontage       462
GarageFinish      129
GarageQual        129
GarageYrBlt       129
GarageCond        129
GarageType        127
BsmtExposure       71
BsmtFinType2       70
BsmtFinType1       69
BsmtCond           69
BsmtQual           69
MasVnrArea         14
MasVnrType         14
BsmtHalfBath        2
BsmtFullBath        2
GarageArea          1
GarageCars          1
Electrical          1
BsmtUnfSF           1
BsmtFinSF2          1
BsmtFinSF1          1
TotalBsmtSF         1
Functional          0
EnclosedPorch       0
PavedDrive          0
dtype: int64

In [5]:
df.isnull().sum().sum()

12254

### Convert Supposingly Numerical Cols To Categorical ###

In [9]:
df[['MSSubClass','YrSold','MoSold']].isnull().sum()

MSSubClass    0
YrSold        0
MoSold        0
dtype: int64

In [10]:
df['MSSubClass'] = df['MSSubClass'].apply(str)
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df['MoSold'].astype(str)

### Object Columns, Fillna with "None" & Dummify! ###

In [13]:
c = df.select_dtypes('object')
c.head(3)

,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,30,RL,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,TA,TA,CBlock,TA,TA,No,Rec,Unf,GasA,TA,Y,SBrkr,TA,Typ,Gd,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,3,2010,WD,Normal
1,120,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,Gd,TA,CBlock,Gd,TA,Mn,GLQ,ALQ,GasA,TA,Y,SBrkr,Gd,Typ,NaN,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,2,2009,WD,Normal
2,30,C (all),Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,Gd,TA,BrkTil,TA,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Detchd,Unf,TA,Po,N,NaN,NaN,NaN,11,2007,WD,Normal


In [14]:
c.shape

(2580, 46)

In [15]:
c.isnull().sum().sum()

11639

In [16]:
c = c.fillna("None")
c.isnull().sum().sum()

0

In [17]:
c.shape

(2580, 46)

In [18]:
c = pd.get_dummies(c, drop_first=True)
c.shape

(2580, 262)

### Numeric Columns, Dropna on Rows/Observations, Create Full (Clean) ###

In [19]:
n = df.select_dtypes('number')
n.head(3)

,Unnamed: 0,PID,GrLivArea,SalePrice,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
0,1,909176150,856,126000,NaN,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0
1,2,905476230,1049,139500,42.0,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0
2,3,911128020,1001,124900,60.0,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0


In [20]:
n.shape

(2580, 36)

In [21]:
n.isnull().sum().sum()

615

In [22]:
full = pd.concat([c, n], axis=1)
full.shape

(2580, 298)

In [23]:
full = full.dropna()
full.shape

(1988, 298)

In [24]:
full.isnull().sum().sum()

0

### MLR For Base Case ###

In [25]:
y=full.SalePrice
X=full.drop('SalePrice', axis=1)

In [26]:
regressor = LinearRegression()
regressor.fit(X,y)

LinearRegression()

In [27]:
regressor.score(X,y)

0.943375217744169

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)

In [29]:
ols = LinearRegression()
ols.fit(X_train, y_train)
print("R^2 for train set: %f" %ols.score(X_train, y_train))

print('-'*50)

print("R^2 for test  set: %f" %ols.score(X_test, y_test))

R^2 for train set: 0.944666
--------------------------------------------------
R^2 for test  set: 0.919440


### Stats for MLR For Base Case ###

In [30]:
# Obtain and set x=input and y=output
X0 = full.drop('SalePrice', axis=1)
y0 = full.SalePrice

In [31]:
#Setup statsmodels.api, inputs (x, x_constant, and y)
x_constant0 = sm.add_constant(X0)

In [32]:
#Run OLS with statsmodels
lin_reg0 = sm.OLS(y0,x_constant0).fit()
lin_reg0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     104.2
Date:                Tue, 19 Apr 2022   Prob (F-statistic):               0.00
Time:                        07:01:25   Log-Likelihood:                -22364.
No. Observations:                1988   AIC:                         4.528e+04
Df Residuals:                    1713   BIC:                         4.682e+04
Df Model:                         274                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -5.818e+05   1.17e+05     -4.987      0.000   -8.11e+05   -3.53e+05
MSSubClass_150           -0.9474      0.227     -4.167      0.000      -1.393      -0.501
MSSubClass_160        -5139.2016   7149.677     -0.719      0.472   -1.92e+04    8883.815
MSSubClass_180         4458.0725   1.06e+04      0.420      0.675   -1.64e+04    2.53e+04
MSSubClass_190         7961.1017   2.68e+04      0.297      0.766   -4.46e+04    6.05e+04
MSSubClass_20          2.035e+04   1.31e+04      1.554      0.120   -5341.490     4.6e+04
MSSubClass_30          2.384e+04   1.35e+04      1.767      0.077   -2618.955    5.03e+04
MSSubClass_40           3.14e+04    1.7e+04      1.850      0.065   -1897.902    6.47e+04
MSSubClass_45          1.419e+04   2.29e+04      0.619      0.536   -3.08e+04    5.91e+04
MSSubClass_50          2.629e+04   1.44e+04      1.819      0.069   -2052.027    5.46e+04
MSSubClass_60          1.937e+04   1.43e+04      1.351      0.177   -8744.047    4.75e+04
MSSubClass_70          2.672e+04   1.45e+04      1.849      0.065   -1621.394    5.51e+04
MSSubClass_75          2.949e+04   1.77e+04      1.662      0.097   -5305.855    6.43e+04
MSSubClass_80          1.375e+04   1.63e+04      0.844      0.399   -1.82e+04    4.57e+04
MSSubClass_85          1.812e+04   1.55e+04      1.165      0.244   -1.24e+04    4.86e+04
MSSubClass_90          3710.3470   7332.021      0.506      0.613   -1.07e+04    1.81e+04
MSZoning_C (all)      -9.058e+04   1.86e+04     -4.881      0.000   -1.27e+05   -5.42e+04
MSZoning_FV           -7.522e+04   1.76e+04     -4.284      0.000    -1.1e+05   -4.08e+04
MSZoning_I (all)      -3.061e+04   1.44e+04     -2.126      0.034   -5.88e+04   -2366.750
MSZoning_RH           -6.693e+04   1.79e+04     -3.740      0.000   -1.02e+05   -3.18e+04
MSZoning_RL           -6.837e+04    1.7e+04     -4.012      0.000   -1.02e+05   -3.49e+04
MSZoning_RM           -7.879e+04   1.75e+04     -4.504      0.000   -1.13e+05   -4.45e+04
Street_Pave            2.458e+04    1.2e+04      2.044      0.041     999.037    4.82e+04
Alley_None             1647.9680   3004.389      0.549      0.583   -4244.690    7540.626
Alley_Pave            -3609.1470   4709.874     -0.766      0.444   -1.28e+04    5628.564
LotShape_IR2           5166.4082   3574.073      1.446      0.148   -1843.599    1.22e+04
LotShape_IR3           4571.3200   8958.699      0.510      0.610    -1.3e+04    2.21e+04
LotShape_Reg           2079.7881   1283.527      1.620      0.105    -437.658    4597.234
LandContour_HLS        1.019e+04   3984.851      2.557      0.011    2375.029     1.8e+04
LandContour_Low       -6579.9503   5792.525     -1.136      0.256   -1.79e+04    4781.218
LandContour_Lvl        5397.3009   2964.724      1.821      0.069    -417.561    1.12e+04
Utilities_NoSewr      -